In [1]:
import pandas as pd
import sqlite3

In [2]:
juul_keywords = pd.read_excel('Juul-Keywords-Original.xlsx')
juul_hashtags = pd.read_excel('#Juul-Original.xlsx')
juul_labeled = pd.read_excel('Juul-Labeled.xlsx')
# Merged keywords and hashtags
frames = [juul_keywords, juul_hashtags]
juul_merged = pd.concat(frames)
juul_merged = juul_merged.drop_duplicates(subset='tweet_id', keep="last")
juul_ml = pd.read_excel('Juul-ML.xlsx')

In [3]:
juul_keywords.shape

(180024, 31)

In [4]:
juul_hashtags.shape

(5452, 31)

In [5]:
juul_merged.shape

(183435, 31)

In [6]:
juul_labeled.shape

(1248, 10)

In [44]:
## Check different data types (e.g. object and int64)
# juul_labeled['tweet_id'] = pd.to_numeric(juul_labeled['tweet_id'])
# print(type(juul_merged['tweet_id'][1]))
# type(juul_merged['tweet_id'][1])
# juul_labeled['tweet_id']
# juul_labeled.dtypes

tweet_id        int64
URL            object
screen_name    object
User           object
Tweet          object
Sentiment      object
Genre          object
Theme          object
Unnamed: 8     object
Unnamed: 9     object
dtype: object

In [51]:
## Separate labeled tweets from the rest (partition)
import numpy as np
juul_merged.sort_values(by = "tweet_id")
juul_merged = juul_merged.reindex()
juul_labeled.sort_values(by = "tweet_id")
juul_labeled = juul_labeled.reindex()

In [52]:
# set differences of two tweet_id columns
diff_array = np.setdiff1d(juul_merged['tweet_id'], juul_labeled['tweet_id'])
# Create a Pandas DataFrame from a Numpy array
diff_df = pd.DataFrame(diff_array, columns = ['tweet_id'])
diff_df = pd.merge(diff_df, juul_merged, how = 'left', left_on = 'tweet_id', right_on = 'tweet_id')
diff_df.shape

(182257, 31)

In [48]:
diff_array

array([ 988790603549954051,  988793004696616960,  988800312159932417, ...,
       1054416806411149313, 1054418864388403200, 1054419887098077189])

In [53]:
# set the intersection of two tweet_id columns
intersect_array = np.intersect1d(juul_merged['tweet_id'], juul_labeled['tweet_id'])
intersect_df = pd.DataFrame(intersect_array, columns = ['tweet_id'])
intersect_df = pd.merge(intersect_df, juul_merged, how = 'left')
intersect_df.shape

(1178, 31)

In [57]:
diff2_array = np.setdiff1d(juul_labeled['tweet_id'], intersect_df['tweet_id'])
diff2_df = pd.DataFrame(diff2_array, columns = ['tweet_id'])
diff2_df.to_excel('diff2.xlsx')

In [6]:
# Since the previous coded tweets have some problems, I have to create a new training (&validation) set in case. 
from sklearn.model_selection import train_test_split
train, test = train_test_split(juul_merged, test_size=0.99)

In [56]:
# Save the new training set as a excel file
# train.to_excel('Juul-Newlabel.xlsx', encoding='utf-8')

In [22]:
# Store Juul tweets in a SQLite database
conn = sqlite3.connect('juul.db')
cur = conn.cursor()
juul_merged.to_sql('juul_merged_table', conn, if_exists='append')
# labeled.to_sql('juul_labeled_table', conn, if_exists='append')
# unlabeled.to_sql('juul_unlabeled_table', conn, if_exists='append')
# juul_ml.to_sql('juul_ml_table', conn, if_exists='append')
pd.read_sql_query("select * from juul_merged_table;", conn)

index  in_reply_to_status_id in_reply_to_screen_name  \
0           0                    NaN                    None   
1           1                    NaN                    None   
2           2                    NaN                    None   
3           3           1.000886e+18              MitchaSays   
4           4                    NaN                    None   
5           5                    NaN                    None   
6           6                    NaN                    None   
7           7                    NaN                    None   
8           8                    NaN                    None   
9           9                    NaN                    None   
10         10                    NaN                    None   
11         11                    NaN                    None   
12         12           1.000877e+18          BlvkSheepMusic   
13         13                    NaN                    None   
14         14                    NaN                    None   
15         15           1.000759e+18         SocialPowerOne1   
16         16                    NaN                    None   
17         17                    NaN                    None   
18         18                    NaN                    None   
19         19           1.001251e+18                 lil_qro   
20         20                    NaN                    None   
21         21                    NaN                    None   
22         22           1.001247e+18          zach58hazelett   
23         23                    NaN                    None   
24         24           1.001209e+18                    Si_G   
25         25                    NaN                    None   
26         26                    NaN                    None   
27         27                    NaN                    None   
28         28           1.001249e+18               taderjoes   
29         29           1.001248e+18           bryan_correa6   
...       ...                    ...                     ...   
183405   5422                    NaN                    None   
183406   5423                    NaN                    None   
183407   5424                    NaN                    None   
183408   5425           1.054208e+18         baileyjones3749   
183409   5426           1.054140e+18           treycetolbert   
183410   5427                    NaN                    None   
183411   5428                    NaN                    None   
183412   5429                    NaN                    None   
183413   5430                    NaN                    None   
183414   5431                    NaN                    None   
183415   5432                    NaN                    None   
183416   5433                    NaN                    None   
183417   5434                    NaN                    None   
183418   5435                    NaN                    None   
183419   5436                    NaN                    None   
183420   5437                    NaN                    None   
183421   5438                    NaN                    None   
183422   5439                    NaN                    None   
183423   5440                    NaN                    None   
183424   5441                    NaN                    None   
183425   5442                    NaN                    None   
183426   5443                    NaN                    None   
183427   5444                    NaN                    None   
183428   5445                    NaN                    None   
183429   5446                    NaN                    None   
183430   5447                    NaN                    None   
183431   5448                    NaN                    None   
183432   5449           1.053475e+18            maddypnapier   
183433   5450                    NaN                    None   
183434   5451                    NaN                    None   

        in_reply_to_user_id  retweet_co

In [5]:
cur.close()
conn.close()